In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

pd.options.display.max_rows = 999
pd.options.display.max_columns = None

# <font color = 'red'> GBM with columns present in data dictionary </font>

In [32]:
dtrain = df_all1.iloc[0:train_rows]
dtest =  df_all1.iloc[train_rows:]

In [33]:
dtest.drop(["is_female"],axis=1,inplace=True)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [131]:
# Building a generic function which does modeli building, cross validation, accuracy metrics calculation, feature importance
# and scoring on the test dataset

def modelfit(alg, dtrain, predictors, dtest, performCV=True, printFeatureImportance=True, cv_folds=5):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['is_female'])
        
    # Scoring on the training dataset
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain['is_female'], cv=cv_folds, scoring='roc_auc')
    
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['is_female'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['is_female'], dtrain_predprob))
    
    if performCV:
        print ("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        
    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
        print(feat_imp)
        #feat_imp.plot(kind='bar', title='Feature Importances')
        #plt.ylabel('Feature Importance Score')

    # Scoring on the test dataset
    test_predprob = pd.DataFrame(alg.predict_proba(dtest))
    
    return test_predprob
    

### Step 0 - Creating the base model

In [36]:
target = 'is_female'
notuse_list = ['AA17', 'RI5_1','RI5_2','RI5_3']

In [199]:
# Running the base model
predictors = [x for x in dtrain.columns if x not in [target, notuse_list]]
gbm0 = GradientBoostingClassifier(random_state=10)
test_ans = modelfit(gbm0, dtrain, predictors, dtest)
test_ans.head(10)

In [1]:
# Creating submission dataset
test_ans[1] .to_csv("submission1.csv",header = ['is_female'], index = True, index_label = 'test_id')
test_ans['is_female'].value_counts()

NameError: name 'np' is not defined

### Step 1 - Fixing learning rate and searching for the right number of trees

In [222]:
param_test1 = {'n_estimators':list(range(50, 1000, 50))}

In [223]:
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.005, min_samples_split=500,\
                                                               min_samples_leaf=50,max_depth=8,max_features='sqrt',\
                                                               subsample=0.8,random_state=10), 
                        param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)


In [224]:
gsearch1.fit(dtrain[predictors],dtrain['is_female'])

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.005, loss='deviance', max_depth=8,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=500, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [225]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.91876, std: 0.00756, params: {'n_estimators': 50},
  mean: 0.92377, std: 0.00662, params: {'n_estimators': 100},
  mean: 0.92613, std: 0.00661, params: {'n_estimators': 150},
  mean: 0.92926, std: 0.00631, params: {'n_estimators': 200},
  mean: 0.93207, std: 0.00583, params: {'n_estimators': 250},
  mean: 0.93448, std: 0.00555, params: {'n_estimators': 300},
  mean: 0.93662, std: 0.00523, params: {'n_estimators': 350},
  mean: 0.93895, std: 0.00505, params: {'n_estimators': 400},
  mean: 0.94109, std: 0.00481, params: {'n_estimators': 450},
  mean: 0.94293, std: 0.00453, params: {'n_estimators': 500},
  mean: 0.94474, std: 0.00442, params: {'n_estimators': 550},
  mean: 0.94627, std: 0.00434, params: {'n_estimators': 600},
  mean: 0.94781, std: 0.00422, params: {'n_estimators': 650},
  mean: 0.94912, std: 0.00415, params: {'n_estimators': 700},
  mean: 0.95037, std: 0.00408, params: {'n_estimators': 750},
  mean: 0.95139, std: 0.00398, params: {'n_estimators': 800},
  mean: 0

## <font color='blue'>Tuning parameters of the tree</font> 

### Step 2 - find the optimum max depth and split size sample for the trees

score keeps on increasing with number of trees. Not much difference when learning rate decreased from 0.1 to 0.05. So, as first step, keep the learning rate constant at 0.1 and number of trees as 100 (because higher learning rate will take lot of time in tuning, so in the end increase the number of trees)

In [231]:
# param_test2 = {'max_depth':list(range(5,16,2)), 'min_samples_split':list(range(200,1001,200))} 
#                -> min samples = 400, max depth = 15
param_test2 = {'max_depth':list(range(10,30,2))} #min_samples fixed at 400
#               -> max depth = 18
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,
                                                               min_samples_split = 400, max_features='sqrt',
                                                               subsample=0.8, random_state=10), 
                        param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

In [232]:
gsearch2.fit(dtrain[predictors],dtrain['is_female'])

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=400, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'max_depth': [10, 12, 14, 16, 18, 20, 22, 24, 26, 28]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [233]:
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: 0.96265, std: 0.00260, params: {'max_depth': 10},
  mean: 0.96272, std: 0.00327, params: {'max_depth': 12},
  mean: 0.96361, std: 0.00311, params: {'max_depth': 14},
  mean: 0.96357, std: 0.00338, params: {'max_depth': 16},
  mean: 0.96391, std: 0.00266, params: {'max_depth': 18},
  mean: 0.96308, std: 0.00328, params: {'max_depth': 20},
  mean: 0.96382, std: 0.00278, params: {'max_depth': 22},
  mean: 0.96358, std: 0.00323, params: {'max_depth': 24},
  mean: 0.96358, std: 0.00391, params: {'max_depth': 26},
  mean: 0.96383, std: 0.00354, params: {'max_depth': 28}],
 {'max_depth': 18},
 0.9639135328249987)

### Step 3 - find the optimum min_samples_leaf

From step 2, max depth of tree set to 18 and min sample split fixed at 400

In [251]:
param_test3 = {'min_samples_leaf':list(range(10,71,10))}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, max_depth=18, 
                                                               max_features='sqrt', min_samples_split = 400,
                                                               subsample=0.8, random_state=10), 
                        param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

In [252]:
gsearch3.fit(dtrain[predictors],dtrain['is_female'])

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=18,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=400, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'min_samples_leaf': [10, 20, 30, 40, 50, 60, 70]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [253]:
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: 0.96314, std: 0.00266, params: {'min_samples_leaf': 10},
  mean: 0.96388, std: 0.00322, params: {'min_samples_leaf': 20},
  mean: 0.96293, std: 0.00320, params: {'min_samples_leaf': 30},
  mean: 0.96310, std: 0.00369, params: {'min_samples_leaf': 40},
  mean: 0.96284, std: 0.00401, params: {'min_samples_leaf': 50},
  mean: 0.96173, std: 0.00331, params: {'min_samples_leaf': 60},
  mean: 0.96221, std: 0.00313, params: {'min_samples_leaf': 70}],
 {'min_samples_leaf': 20},
 0.9638819706163682)

### Step 4 - find the optimum max_features

From step 3, min sample leaf of tree set to 20

In [263]:
# param_test4 = {'max_features':list(range(7,20,2))}
# param_test4 = {'max_features':list(range(15,50,5))}
# param_test4 = {'max_features':list(range(50,200,10))} # -> best - 80
# param_test4 = {'max_features':list(range(100,301,25))}
param_test4 = {'max_features':list(range(250,750,50))}

gsearch4 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,max_depth=18, 
                                                               min_samples_split=400, min_samples_leaf=20, 
                                                               subsample=0.8, random_state=10),
                        param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

In [264]:
gsearch4.fit(dtrain[predictors],dtrain['is_female'])

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=18,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=20,
              min_samples_split=400, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'max_features': [250, 300, 350, 400, 450, 500, 550, 600, 650, 700]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [265]:
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: 0.96958, std: 0.00244, params: {'max_features': 250},
  mean: 0.96996, std: 0.00232, params: {'max_features': 300},
  mean: 0.96985, std: 0.00294, params: {'max_features': 350},
  mean: 0.97012, std: 0.00242, params: {'max_features': 400},
  mean: 0.97025, std: 0.00280, params: {'max_features': 450},
  mean: 0.97019, std: 0.00259, params: {'max_features': 500},
  mean: 0.97028, std: 0.00261, params: {'max_features': 550},
  mean: 0.97005, std: 0.00296, params: {'max_features': 600},
  mean: 0.96994, std: 0.00251, params: {'max_features': 650},
  mean: 0.97019, std: 0.00239, params: {'max_features': 700}],
 {'max_features': 550},
 0.9702771499868742)

## <font color = 'Blue'> Tuning parameters of Boosting </font> 

### Step 5 - find the optimum subsample
(fraction of observations to be selected for each tree, 0.8 is recommended. Reducing it below 1 helps take care of variance)

From step 4, max_features set to 550

In [267]:
param_test5 = {'subsample':[0.6,0.65,0.7,0.75,0.8,0.85,0.9,1]}
# Chosing 0.8 though 1 gives highest since 1 would mean the whole sample eevry time and that would reduce the variance
# 0.8 was the 2nd highest
gsearch5 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,max_depth=18,
                                                               min_samples_split=400, min_samples_leaf=20, 
                                                               random_state=10,max_features=550),
param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

In [268]:
gsearch5.fit(dtrain[predictors],dtrain['is_female'])

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=18,
              max_features=550, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=20,
              min_samples_split=400, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=10,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'subsample': [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [269]:
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

([mean: 0.96912, std: 0.00305, params: {'subsample': 0.6},
  mean: 0.96965, std: 0.00223, params: {'subsample': 0.65},
  mean: 0.96979, std: 0.00249, params: {'subsample': 0.7},
  mean: 0.97020, std: 0.00273, params: {'subsample': 0.75},
  mean: 0.97028, std: 0.00261, params: {'subsample': 0.8},
  mean: 0.97024, std: 0.00276, params: {'subsample': 0.85},
  mean: 0.97015, std: 0.00281, params: {'subsample': 0.9},
  mean: 0.97047, std: 0.00260, params: {'subsample': 1}],
 {'subsample': 1},
 0.9704667042838306)

In [39]:
# LB score = 0.90884 (no increase after including this tuned parameters. Same as the gbm0)

predictors = [x for x in dtrain.columns if x not in [target, notuse_list]]
gbm1 = GradientBoostingClassifier(learning_rate=0.1,
                                  n_estimators=100,
                                  max_depth=18,
                                  min_samples_split=400, 
                                  min_samples_leaf=20, 
                                  random_state=10,
                                  max_features=550,
                                  subsample = 0.8)
test_ans1 = modelfit(gbm1, dtrain, predictors, dtest)

In [41]:
test_ans1[1] .to_csv("submission2a.csv",header = ['is_female'], index = True, index_label = 'test_id')
test_ans1['is_female'].value_counts()

### Step 6 - tuning learning rate and number of trees
Reduce the learning rate and increase the number of trees at the same time to see if that increases the score

In [45]:
predictors = [x for x in dtrain.columns if x not in [target, notuse_list]]
gbm2 = GradientBoostingClassifier(learning_rate=0.01,
                                  n_estimators=500,
                                  max_depth=18,
                                  min_samples_split=400, 
                                  min_samples_leaf=20, 
                                  random_state=10,
                                  max_features=550,
                                  subsample = 0.8)
test_ans2 = modelfit(gbm2, dtrain, predictors, dtest)

In [47]:
test_ans2['is_female'] = np.where(test_ans2[0]>test_ans2[1],0,1)
test_ans2['is_female'] .to_csv("submission2b.csv",header = ['is_female'], index = True, index_label = 'test_id')
test_ans2['is_female'].value_counts()

In [135]:
predictors = [x for x in dtrain.columns if x not in [target, notuse_list]]
gbm3 = GradientBoostingClassifier(learning_rate=0.1,
                                  n_estimators=1000,
                                  max_depth=18,
                                  min_samples_split=400, 
                                  min_samples_leaf=20, 
                                  random_state=10,
                                  max_features=550,
                                  subsample = 0.8)
test_ans3 = modelfit(gbm3, dtrain, predictors, dtest)

In [138]:
gbm_dict_prob1 = pd.DataFrame()
gbm_dict_prob1['is_female'] = test_ans3[1]
gbm_dict_prob1 .to_csv("gbm_dict_prob1.csv",index = True, index_label = 'test_id')

,is_female
0,9.999848e-01
1,1.023028e-03
2,9.989044e-01
3,9.999539e-01
4,8.343746e-02
5,9.999771e-01
6,3.587314e-04
7,9.999842e-01
8,9.622705e-07
9,9.005619e-03


# <font color = 'red'> GBM with all columns </font>

In [18]:
dtrain3 = df_all3.iloc[0:train_rows]
dtest3 =  df_all3.iloc[train_rows:]

In [32]:
# Dont do this step for XGBoost
dtest3.drop(["is_female","train_id"],axis=1,inplace=True)
dtrain3.drop(["test_id"],axis=1,inplace=True)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
# Base model
target = 'is_female'
notuse_list = ['AA17', 'RI5_1','RI5_2','RI5_3', 'train_id']
predictors = [x for x in dtrain3.columns if x not in [target, notuse_list]]

gbm_all_0 = GradientBoostingClassifier(random_state=10)

test_ans4 = modelfit(gbm_all_0, dtrain2, predictors, dtest2)

In [40]:
# Model with the same final parameters as the model with only the columns in data dictionary
gbm_all_3 = GradientBoostingClassifier(learning_rate=0.1,
                                  n_estimators=1000,
                                  max_depth=18,
                                  min_samples_split=400, 
                                  min_samples_leaf=20, 
                                  random_state=10,
                                  max_features=550,
                                  subsample = 0.8)
test_ans_all3 = modelfit(gbm_all_3, dtrain3, predictors, dtest3)

In [54]:
gbm_all_prob1 = pd.DataFrame()
gbm_all_prob1['is_female'] = test_ans_all3[1]
gbm_all_prob1 .to_csv("submission_gbm_all_prob1.csv",index = True, index_label = 'test_id')

,is_female
0,9.999605e-01
1,5.593367e-04
2,9.969051e-01
3,9.999582e-01
4,6.681739e-02
5,9.999778e-01
6,9.970381e-03
7,9.999864e-01
8,2.819521e-06
9,1.051219e-01
